In [1]:
import pandas as pd
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
import pyreadstat

In [2]:
df, meta = pyreadstat.read_dta('Combined_a160c.dta')

In [3]:
meta.max_rows = 4000

In [4]:
meta.column_labels

['XW Cross wave ID',
 'DV: randomised household id',
 'HF Person number',
 'DV: Randomised person id',
 'HF State',
 'DV: Randomised original area id',
 'HQ Date of interview - Household Questionnaire',
 'HF Total contacts',
 'HF Final household response status',
 'HF Number of in-scope persons in household',
 'HF1 Cross wave ID (xwaveid) - 0001',
 'HF1 Cross wave ID (xwaveid) - 0002',
 'HF1 Cross wave ID (xwaveid) - 0003',
 'HF1 Cross wave ID (xwaveid) - 0004',
 'HF1 Cross wave ID (xwaveid) - 0005',
 'HF1 Cross wave ID (xwaveid) - 0006',
 'HF1 Cross wave ID (xwaveid) - 0007',
 'HF1 Cross wave ID (xwaveid) - 0008',
 'HF1 Cross wave ID (xwaveid) - 0009',
 'HF1 Cross wave ID (xwaveid) - 0010',
 'HF1 Cross wave ID (xwaveid) - 0011',
 'HF1 Cross wave ID (xwaveid) - 0012',
 'HF1 Cross wave ID (xwaveid) - 0013',
 'HF1 Cross wave ID (xwaveid) - 0014',
 'HF1 Cross wave ID (xwaveid) - 0015',
 'HF1 Cross wave ID (xwaveid) - 0016',
 'HF1 Cross wave ID (xwaveid) - 0017',
 'HF1 Cross wave ID (xwave

In [5]:
meta_dict = dict(zip(meta.column_names, meta.column_labels))


In [6]:
meta_dict['ajbprhr']

'E7 Total hours per week would choose to work'

In [7]:
meta_dict['aesbrd']

'DV: Current labour force status - broad'

In [ ]:
overworked = list_dfs[8][list_dfs[8]['ijbhrcpr'] == '[1] Fewer hours']
pref = overworked['ijbhruc']

In [ ]:
(pref.value_counts(normalize=True)*100).round(2)

In [ ]:
list_df11  = pd.read_stata('Combined_o160c.dta')

In [ ]:
overworked = list_df11[list_df11['ojbhrcpr'] == '[1] Fewer hours']
pref = overworked['ojbhruc']
(pref.value_counts(normalize=True)*100).round(2)

In [ ]:
pd.to_numeric(list_dfs[0]['ohgage'], errors = 'coerce').mean()

In [66]:
filenames = glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p]*160c.dta')
list_dfs  = [pd.read_stata(f) for f in filenames]

In [67]:
waves_jbhr_index = [list_dfs[i].filter(regex='jbhrcpr$').columns for i in range(16)] #This finds the index for the string 'jbhrcpr' in all waves
waves_jbhr = [list_dfs[i][waves_jbhr_index[i]] for i in range(16)] #waves_jbhr[0-10] are now created (waves1-11)
waves_sex_index = [list_dfs[i].filter(regex='hgsex$').columns for i in range(16)] #Finds the index for sex in each dataset
waves_sex = [list_dfs[i][waves_sex_index[i]] for i in range(16)] #waves_sex[0-10] are now created (waves1-11)

waves_employed_index = [list_dfs[i].filter(regex='esbrd$').columns for i in range(16)] #
waves_employed = [list_dfs[i][waves_employed_index[i]] for i in range(16)] #waves_employed[0-10] are now created (waves1-11)

waves_join = [waves_jbhr[i].join([waves_sex[i],waves_employed[i]]) for i in range(16)] #This joins the columns of jbhr and sex for each data

In [70]:
waves_excluded = [waves_join[i][waves_join[i].iloc[:,2] == '[1] Employed'] for i in range(16)]
waves_excluded = [waves_excluded[i].iloc[:,:2] for i in range(16)]
waves = [waves_excluded[i].groupby([waves_excluded[i].iloc[:,0],waves_excluded[i].iloc[:,1]]).size().unstack() for i in range(16)]
waves = [waves[i].T for i in range(16)]


In [71]:
total = [waves[i].sum(axis=1) for i in range(16)]
total_df = [pd.DataFrame(total[i]) for i in range(16)]
total_combine = [total_df[i].join(waves[i]) for i in range(16)]

In [72]:
frac_waves = [waves[i].div(waves[i].sum(axis=1), axis=0) * 100 for i in range(16)]

In [73]:
total_combine = [total_df[i].join(frac_waves[i]) for i in range(16)]
total_combine = [total_combine[i].rename(columns={0:'Total Employed'}) for i in range(16)]
total_combine = [total_combine[i].fillna(0) for i in range(16)]

In [74]:
final_table = total_combine[0].merge(total_combine[1], how = 'outer')
final_table = final_table.merge(total_combine[2], how = 'outer')
final_table = final_table.merge(total_combine[3], how = 'outer')
final_table = final_table.merge(total_combine[4], how = 'outer')
final_table = final_table.merge(total_combine[5], how = 'outer')
final_table = final_table.merge(total_combine[6], how = 'outer')
final_table = final_table.merge(total_combine[7], how = 'outer')
final_table = final_table.merge(total_combine[8], how = 'outer')
final_table = final_table.merge(total_combine[9], how = 'outer')
final_table = final_table.merge(total_combine[10], how = 'outer')
final_table = final_table.merge(total_combine[11], how = 'outer')
final_table = final_table.merge(total_combine[12], how = 'outer')
final_table = final_table.merge(total_combine[13], how = 'outer')
final_table = final_table.merge(total_combine[14], how = 'outer')
final_table = final_table.merge(total_combine[15], how = 'outer')

In [75]:
final_table = final_table.fillna('')
row_list = final_table.index
num_rows = len(row_list)

In [76]:
new_rows = []
for i in range(num_rows):
    if i == 0 or i %2 == 0:
        new_rows.append('Male')
    else:
        new_rows.append('Female')

In [78]:
waves = []
for i in range(1,17):
    if i == 0 or i %2 == 0:
        waves.append('Wave {}'.format(i))
        waves.append('Wave {}'.format(i))
    else:
        waves.append('Wave {}'.format(i))
        waves.append('Wave {}'.format(i))

In [81]:
final_rows = [waves[i] + ' ' + new_rows[i] for i in range(32)]
final_table.index = new_rows

In [82]:
a = 0
b = 2
final_locs = []
while a < 31:
    final_locs.append(final_table.iloc[a:b,:])
    a = a + 2
    b = b + 2

In [83]:
final_table = final_table.reset_index()

In [87]:
waves = []
for i in range(1,17):
    if i == 0 or i %2 == 0:
        waves.append('Wave_{}'.format(i))
        waves.append('Wave_{}'.format(i))
    else:
        waves.append('Wave_{}'.format(i))
        waves.append('Wave_{}'.format(i))

In [89]:
final_table.index = waves
final_table = final_table.rename(columns={'index':'Sex'})

In [90]:
final_table.reset_index(inplace = True)

In [91]:
final_table.set_index(['index', 'Sex'], inplace=True)

In [92]:
final_table = final_table.round(1)

In [93]:
Table_1 = final_table[['Total Employed', '[1] Fewer hours', '[3] More hours']]
Table_1.columns = [['All Employed', 'Prefer to work less (%)', 'Prefer to work more (%)']]

In [94]:
Table_1.reset_index(inplace=True)

In [95]:
Table_1m = Table_1[Table_1.iloc[:,1] == 'Male']
Table_1m.reset_index(inplace=True)
Table_1m = Table_1m.iloc[:,1:]

In [96]:
Table_1f = Table_1[Table_1.iloc[:,1] == 'Female']
Table_1f.reset_index(inplace=True)
Table_1f = Table_1f.iloc[:,2:]

In [97]:
Table_1 = Table_1m.join(Table_1f, lsuffix='sex')

In [101]:
years_list = []
for i in range(1,10):
    years = '200{}'.format(i)
    years_list.append(years)
years_list = years_list + ['2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [102]:
years_list

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016']

In [103]:
Table_1.index=years_list

In [104]:
Table_1.rename(columns={'index':'Waves'}, inplace=True)

In [105]:
Table_1m.index = years_list
Table_1m = Table_1m.iloc[:,1:]

In [106]:
Table_1f.index = years_list

In [107]:
new_table = Table_1m.append(Table_1f)

In [108]:
new_table.index.name = 'Wave'

In [109]:
new_table

,Sex,All Employed,Prefer to work less (%),Prefer to work more (%)
Wave,,,,
2001,Male,4549,30.2,15.2
2002,Male,4325,31.3,15.1
2003,Male,4247,31.3,13.7
2004,Male,4155,30.9,13.3
2005,Male,4304,29.6,13.2
2006,Male,4353,29.6,13.1
2007,Male,4308,28.7,12.0
2008,Male,4337,27.8,12.2
2009,Male,4493,27.4,14.1


In [119]:
new_table.groupby(new_table.index)['Sex'].size()

TypeError: only integer scalar arrays can be converted to a scalar index

In [122]:
new_table.columns

MultiIndex([(                'Sex',),
            (       'All Employed',),
            ('Prefer to work less',),
            ('Prefer to work more',)],
           )

In [124]:
new_table.columns = [['Sex', 'All Employed', 'Prefer to work less (%)', 'Prefer to work more (%)']]

In [127]:
new_table.columns

MultiIndex([(                    'Sex',),
            (           'All Employed',),
            ('Prefer to work less (%)',),
            ('Prefer to work more (%)',)],
           )